In [0]:
from pyspark.sql.functions import *

  # Azure Event Hub Configuration
event_hub_namespace='hospital-analytics-mn'
event_hub_name='hospital-analytis-eh'
event_hub_conn_str=dbutils.secrets.get(scope='hospitalAnalyticsVaultScope', key='eventhub-connection-string')

kafka_options = {
    'kafka.bootstrap.servers':f'{event_hub_namespace}.servicebus.windows.net:9093',
      'subscribe': event_hub_name,
          'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
          'kafka.sasl.mechanism': 'PLAIN',
          'kafka.security.protocol':  'SASL_SSL',
          'startingOffsets': 'latest',
          'failOnDataLoss': 'false'
}

# Read form event hub
raw_df = (spark.readStream.format('kafka')
  .options(**kafka_options)
  .load()
)

# Cast data to json
json_df = raw_df.selectExpr('CAST(value AS STRING) as raw_json')

# ADLS configuration
# We can use either key or string based access to accees the azure storage account (Using key based access)
spark.conf.set( 
               "fs.azure.account.key.hospitalstorage224.dfs.core.windows.net",
  dbutils.secrets.get(scope='hospitalAnalyticsVaultScope', key='storage-connection'))

bronze_path = "abfss://bronze@hospitalstorage224.dfs.core.windows.net/patient_flow"

# Write stream to bronze
(
    json_df
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/mnt/bronze/_checkpoints/patient_flow")
    .start(bronze_path)
)

In [0]:
df_bronze=spark.read.format('delta').load(bronze_path)
display(df_bronze.limit(10))

raw_json
"{""patient_id"": ""c6e64a4e-693e-46b1-b911-33df8ce84e08"", ""gender"": ""Female"", ""age"": 20, ""department"": ""Emergency"", ""admission_time"": ""2025-12-13T16:15:56.867738"", ""discharge_time"": ""2025-12-14T10:15:56.869584"", ""bed_id"": 432, ""hospital_id"": 6, ""event_timestamp"": ""2025-12-14T17:15:56.869584"", ""event_type"": ""discharge""}"
"{""patient_id"": ""18786d77-bfc4-4b55-980f-19668ef7f52a"", ""gender"": ""Female"", ""age"": 35, ""department"": ""Maternity"", ""admission_time"": ""2025-12-13T18:16:15.891394"", ""discharge_time"": ""2025-12-15T00:16:15.891394"", ""bed_id"": 441, ""hospital_id"": 2, ""event_timestamp"": ""2025-12-14T17:16:15.891394"", ""event_type"": ""discharge""}"
"{""patient_id"": ""6303623f-eadd-47c3-944d-4590691d62a6"", ""gender"": ""Female"", ""age"": 35, ""department"": ""Cardiology"", ""admission_time"": ""2025-12-13T06:09:57.691021"", ""discharge_time"": ""2025-12-14T14:09:57.691021"", ""bed_id"": 331, ""hospital_id"": 6, ""event_timestamp"": ""2025-12-14T17:09:57.691021"", ""event_type"": ""transfer""}"
"{""patient_id"": ""438a79a8-f22b-4ac6-990a-95995ed4fcd6"", ""gender"": ""Other"", ""age"": 10, ""department"": ""Pediatrics"", ""admission_time"": ""2025-12-13T10:08:03.073721"", ""discharge_time"": ""2025-12-13T12:08:03.073721"", ""bed_id"": 118, ""hospital_id"": 3, ""event_timestamp"": ""2025-12-14T17:08:03.073721"", ""event_type"": ""admission""}"
"{""patient_id"": ""2629081d-e502-4034-86ef-dd64a9538e50"", ""gender"": ""Other"", ""age"": 71, ""department"": ""Pediatrics"", ""admission_time"": ""2025-12-11T17:16:12.999343"", ""discharge_time"": ""2025-12-12T04:16:12.999343"", ""bed_id"": 211, ""hospital_id"": 5, ""event_timestamp"": ""2025-12-14T17:16:12.999343"", ""event_type"": ""admission""}"
"{""patient_id"": ""4c209e32-db4a-4c2d-8ccb-86cfede41ff8"", ""gender"": ""Other"", ""age"": 98, ""department"": ""Surgery"", ""admission_time"": ""2025-12-11T19:06:34.788312"", ""discharge_time"": ""2025-12-13T21:06:34.789055"", ""bed_id"": 183, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-14T17:06:34.789055"", ""event_type"": ""transfer""}"
"{""patient_id"": ""1555c6e1-172d-4c85-ad3d-c3b5e2000da2"", ""gender"": ""Other"", ""age"": 66, ""department"": ""ICU"", ""admission_time"": ""2025-12-14T09:11:36.278337"", ""discharge_time"": ""2025-12-16T21:11:36.278337"", ""bed_id"": 152, ""hospital_id"": 4, ""event_timestamp"": ""2025-12-14T17:11:36.278337"", ""event_type"": ""admission""}"
"{""patient_id"": ""98d8c5c1-87f3-4009-baf8-d259cbbb0f53"", ""gender"": ""Male"", ""age"": 59, ""department"": ""ICU"", ""admission_time"": ""2025-12-11T23:17:38.536794"", ""discharge_time"": ""2025-12-13T05:17:38.536794"", ""bed_id"": 284, ""hospital_id"": 1, ""event_timestamp"": ""2025-12-14T17:17:38.536794"", ""event_type"": ""discharge""}"
"{""patient_id"": ""111079fa-1cd3-409d-9924-6ee93bb919f0"", ""gender"": ""Male"", ""age"": 69, ""department"": ""Surgery"", ""admission_time"": ""2025-12-14T15:06:56.180165"", ""discharge_time"": ""2025-12-15T20:06:56.180165"", ""bed_id"": 224, ""hospital_id"": 6, ""event_timestamp"": ""2025-12-14T17:06:56.180165"", ""event_type"": ""transfer""}"
"{""patient_id"": ""e81d8c53-e7ec-4b2d-ab67-2524cf1d6d73"", ""gender"": ""Male"", ""age"": 45, ""department"": ""Surgery"", ""admission_time"": ""2025-12-14T12:06:58.098952"", ""discharge_time"": ""2025-12-15T20:06:58.098952"", ""bed_id"": 421, ""hospital_id"": 3, ""event_timestamp"": ""2025-12-14T17:06:58.098952"", ""event_type"": ""admission""}"
